# Semantic search quick start

This interactive notebook will introduce you to some basic operations with Elasticsearch, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
You'll perform semantic search using [Sentence Transformers](https://www.sbert.net) for text embedding. Learn how to integrate traditional text-based search with semantic search, for a hybrid search system.


## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Since we're running Elasticsearch locally via Docker, we'll connect using the hostname and credentials.


In [1]:
import os
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer


# Setup the Embedding Model

For this example, we're using `all-MiniLM-L6-v2`, part of the `sentence_transformers` library. You can read more about this model on [Huggingface](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).


In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), connecting to the local Elasticsearch instance running in Docker.


In [4]:
# Get credentials from environment variables (set by docker-compose)
ES_HOST = os.getenv('ELASTICSEARCH_HOST', 'http://elasticsearch:9200')
ES_USERNAME = os.getenv('ELASTICSEARCH_USERNAME', 'elastic')
ES_PASSWORD = os.getenv('ELASTICSEARCH_PASSWORD', 'fGIJjohi')

# Create the client instance
client = Elasticsearch(
    hosts=[ES_HOST],
    basic_auth=(ES_USERNAME, ES_PASSWORD),
    verify_certs=False
)


### Test the Client
Before you continue, confirm that the client has connected with this test.


In [5]:
print(client.info())


{'name': '7e1e1af543c4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tnnT_HFSSayyDRuqAS88xg', 'version': {'number': '9.2.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ed771e6976fac1a085affabd45433234a4babeaf', 'build_date': '2025-11-27T08:06:51.614397514Z', 'build_snapshot': False, 'lucene_version': '10.3.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


## Example: Create an Index and Perform Semantic Search

Now you can start using Elasticsearch! Here's a simple example to get you started.


In [6]:
# Create an index with a dense vector field
index_name = "test_index"

# Delete index if it exists
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)

# Create index with mapping for dense vector
client.indices.create(
    index=index_name,
    mappings={
        "properties": {
            "title": {"type": "text"},
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,  # all-MiniLM-L6-v2 produces 384-dimensional vectors
                "index": True,
                "similarity": "cosine"
            }
        }
    }
)

print(f"Index '{index_name}' created successfully!")


Index 'test_index' created successfully!


In [7]:
# Index some sample documents
documents = [
    {"title": "Introduction to Python programming"},
    {"title": "Advanced machine learning techniques"},
    {"title": "Elasticsearch search and analytics"},
    {"title": "Docker containerization guide"}
]

for i, doc in enumerate(documents):
    # Generate embedding for the title
    doc["title_vector"] = model.encode(doc["title"]).tolist()

    # Index the document
    client.index(index=index_name, id=i+1, document=doc)

# Refresh the index to make documents searchable
client.indices.refresh(index=index_name)
print(f"Indexed {len(documents)} documents")


Indexed 4 documents


In [8]:
# Perform semantic search
query = "programming languages"
query_vector = model.encode(query).tolist()

response = client.search(
    index=index_name,
    knn={
        "field": "title_vector",
        "query_vector": query_vector,
        "k": 3,
        "num_candidates": 10
    }
)

print(f"Search results for '{query}':\n")
for hit in response["hits"]["hits"]:
    print(f"Title: {hit['_source']['title']}")
    print(f"Score: {hit['_score']:.4f}\n")


Search results for 'programming languages':

Title: Introduction to Python programming
Score: 0.7557

Title: Advanced machine learning techniques
Score: 0.6019

Title: Elasticsearch search and analytics
Score: 0.5348

